In [1]:
from scipy.stats import norm
from scipy.stats import beta
from scipy.stats import gamma
from scipy.stats import chi2

import time as time
import csv
import pickle
import datetime

import autograd.numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from confint import confint

In [2]:
# data generation parameters
n = 1000
dist = "gamma"
# dist = "gaussian"
# dist = "chisq"
# df = 3
# dist = "uniform"

#confidence level 1 - alpha
alpha = 0.1

#number of threads
thread_num = 40

#algorithm parameters
opt_pts_ratio = 0.3 #subsampling ratio from design points to optimize over
mu = 8.0#4.0(uniform)
M = 7.0#(gamma)#12.0(gaussian)#5.0(uniform)#8.0(chisq)
tau_max = 1e3#1e5#
max_iters = 50
min_iters = 15

In [3]:
repeat_times = 1000

stuff = {}
is_save = True

run_time_ave = 0
num_nan_repeat = 0
num_acc_nan_repeat = 0
now = datetime.datetime.now()
mmddyyhhmm = ("%d_%d_%d_%d_%d" % (now.month, now.day, now.year, now.hour, now.minute))
part_of_out_fn = dist + "_n_" + str(n) + "_uid_" + mmddyyhhmm

np.random.seed(1)

In [4]:
for repeat in range(repeat_times):
    if (dist == "gaussian"):
        X = np.random.randn(n)
    elif (dist == "gamma"):
        X = np.random.gamma(shape=1.0, size=n)
    elif (dist == "chisq"):
        X = np.random.chisquare(df=df, size=n)
    elif(dist == "uniform"):
        X = np.random.uniform(low=-10, high=10, size=n)
    elif(dist == "mixture"):
        n_minus1 = n / 2
        n_plus1 = n - n_minus1
        X = np.concatenate([-2 + np.random.randn(n_minus1), 2 + np.random.randn(n_plus1)])
    else:
        print("ERROR: unsupported distribution")

    X = np.sort(X)

    conf_int = confint(n, X, alpha, opt_pts_ratio=opt_pts_ratio)
    t0 = time.time()
    conf_int.compute_pw_conf_ints(thread_num=thread_num, M=M, tau_max=tau_max, mu=mu, 
                                  max_iters=max_iters, min_iters=min_iters, want_verbose=False)
    
    t1 = time.time()
    
    run_time_ave += (t1 - t0) / repeat_times
    num_acc_nan_repeat += conf_int.num_nans
    if conf_int.num_nans != 0:
        num_nan_repeat += 1
    
    if len(conf_int.idxes_of_design_pts_to_opt) <= 1:
        continue

    stuff[dist + str(repeat) + "_sample_pts"] = X
    stuff[dist + str(repeat) + "_design_pts"] = X[conf_int.idxes_of_design_pts]
    stuff[dist + str(repeat) + "_opt_design_pts"] = X[conf_int.idxes_of_design_pts_to_opt]
    stuff[dist + str(repeat) + "_lo"] = conf_int.lo_opt_pts
    stuff[dist + str(repeat) + "_hi"] = conf_int.hi_opt_pts
    stuff[dist + str(repeat) + "_improved_lo"] = conf_int.improved_lo_opt_pts
    
    stuff[dist + str(repeat) + "_extended_x"] = conf_int.extended_x
    stuff[dist + str(repeat) + "_extended_lo"] = conf_int.improved_lo_extended_pts
    stuff[dist + str(repeat) + "_extended_hi"] = conf_int.improved_hi_extended_pts 
    
    stuff[dist + str(repeat) + "_run_time"] = t1 - t0
    stuff[dist + str(repeat) + "_failure_num"] = conf_int.num_nans
    stuff[dist + str(repeat) + "_failure_design_pts"] = conf_int.failure_design_pts
    
    print("trial", repeat, "run_time", t1 - t0, "failure_num", conf_int.num_nans, "failure_design_pts", conf_int.failure_design_pts)
    

num design points to optimize 39


/Users/xinyueshen/anaconda3/lib/python3.7/site-packages/cvxpy/reductions/solvers/solving_chain.py:163: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)
/Users/xinyueshen/anaconda3/lib/python3.7/site-packages/cvxpy/reductions/solvers/solving_chain.py:163: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)
/Users/xinyueshen/anaconda3/lib/python3.7/site-packages/cvxpy/

trial 0 run_time 255.41604495048523 failure_num 0 failure_design_pts []


In [5]:
print("number of total nans", num_acc_nan_repeat)
print("number of repetitions containing nans", num_nan_repeat)
print("ave run time", run_time_ave)

number of total nans 0
number of repetitions containing nans 0
ave run time 255.41604495048523


In [6]:
if is_save:
    pickle.dump(stuff, open("data_%s.pkl" % (part_of_out_fn), "wb"))